# Demo notebook for interactive plotting
SSC, September 2022

This demonstrates usage of the interactive plotting methods.

In [ ]:
# Please ignore this cell: extra install steps that are only executed when running the notebook on Google Colab
# flake8-noqa-cell
import os
if 'google.colab' in str(get_ipython()) and not os.path.isdir('Test_Data'):
    # we're running on colab and we haven't already downloaded the test data
    # first install pinned version of setuptools (latest version doesn't seem to work with this package on colab)
    !pip install setuptools==61 -qqq
    # install the moralization package
    !pip install git+https://github.com/ssciwr/moralization.git -qqq
    # download test data sets
    !wget https://github.com/ssciwr/moralization/archive/refs/heads/test_data.zip -q
    !unzip -qq test_data.zip && mv -f moralization-test_data/*_Data . && rm -rf moralization-test_data test_data.zip

In [ ]:
import moralization as mn
import moralization.plot as mnplot

In [ ]:
data_dict = mn.InputOutput.get_input_dir("All_Data/XMI_11")

In [ ]:
heatmap = mnplot.InteractiveCategoryPlot(data_dict)

In [ ]:
heatmap.show()